演習課題：サイコロの目の認識AIの構築

dice-02.ipynb : 画像からサイコロ領域を切り出してカラー画像と白黒画像で保存

アスペクト比は1:1で画像のリサイズ処理あり

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# OpenCV imshowサポートパッチのインポート
from google.colab.patches import cv2_imshow

# ライブラリインポート
import os # ファイル操作関係
import re # 文字列処理
import cv2 # 画像処理
import numpy as np # 配列処理
from scipy import stats # 統計処理

Mounted at /content/drive


保存する画像のサイズは size_w, size_hで指定してください

白黒画像に変換する抽出パラメータ（HSV）は lower, upperで指定できます
0 < h < 180, 0 < s, v < 255

In [ ]:
# 保存する画像サイズ（モデルへの入力サイズ）
size_w = 32     # width
size_h = 32     # height

# 画像2値化パラメータ（デフォルトは彩度で調整）
# HSV 抽出下限値（h,s,v）
lower = np.array([0,80,0])
# HSV 抽出上限値（h,s,v）
upper = np.array([180,255,255])

# 読み込み画像枚数カウント
dice1_count = 0
dice2_count = 0
dice3_count = 0
dice4_count = 0
dice5_count = 0
dice6_count = 0

# 読み込みフォルダパス
directory_name = '/content/drive/MyDrive/Colab Notebooks/AI-Kouza/dataset/img_dice/'
# フォルダ内のファイル名を全て取得
file_list = os.listdir(directory_name)
print(file_list)
print('File number: {}'.format(len(file_list)))
# データ一時保管用配列
list_img_crop_bw = np.empty((len(file_list), size_h, size_w), dtype=np.uint8)
list_img_crop_rgb = np.empty((len(file_list), size_h, size_w, 3), dtype=np.uint8)
list_img_filename = np.empty((len(file_list), 1), dtype=object)

# 保存先フォルダが存在しなかったら新規作成
# 白黒
directory_save_bw = '/content/drive/MyDrive/Colab Notebooks/AI-Kouza/dataset/img_dice_bw/'
if not os.path.exists(directory_save_bw):
  os.mkdir(directory_save_bw)
# カラー
directory_save_rgb = '/content/drive/MyDrive/Colab Notebooks/AI-Kouza/dataset/img_dice_rgb/'
if not os.path.exists(directory_save_rgb):
  os.mkdir(directory_save_rgb)



# サイコロ領域をクロップする処理（アスペクト比，サイズ変更含む）
#白黒画像，カラー画像の両方を生成
def cropimage(img_bw, img_raw):
  # クロップ処理
  nonzero = cv2.findNonZero(img_bw) # 白いピクセルの座標リスト取得
  ref_x, ref_y, width, height = cv2.boundingRect(nonzero) # 白い領域を含む矩形座標取得
  img_nonzero = img_bw[ref_y:ref_y+height, ref_x:ref_x+width] # クロップ
  img_raw_crop = img_raw[ref_y:ref_y+height, ref_x:ref_x+width, :]
  # 正方形化処理
  im_size = max(width, height) # 画像縦横の大きい方のサイズを取得
  img_1x1 = np.zeros((im_size, im_size), dtype=np.uint8) # 正方形に変換するベース
  img_1x1_rgb = np.zeros((im_size, im_size, 3), dtype=np.uint8)
  if width > height: # 1:1の画像へ変換する処理
    offset_x = 0
    offset_y = int((width-height)/2)
    img_1x1[offset_y:offset_y+height, :] = img_nonzero
    img_1x1_rgb[offset_y:offset_y+height , :, :] = img_raw_crop
  else:
    offset_x = int((height-width)/2)
    offset_y = 0
    img_1x1[:, offset_x:offset_x+width] = img_nonzero
    img_1x1_rgb[:, offset_x:offset_x+width, :] = img_raw_crop
  # 回転処理のための余白を画像周囲に生成（ゼロパディング）
  pad = int((im_size * 1.2) / 2)
  img_1x1 = cv2.copyMakeBorder(img_1x1, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(0,0,0))
  img_1x1_rgb = cv2.copyMakeBorder(img_1x1_rgb, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=(0,0,0))
  # リサイズ (size_x, size_y)
  img_resize = cv2.resize(img_1x1, (size_w, size_h))
  img_resize_raw = cv2.resize(img_1x1_rgb, (size_w, size_h))
  # 返り値
  return img_resize, img_resize_raw



# メイン処理
# 画像データ読み込み
j=0
for i, file_name in enumerate(file_list):
  if re.search('\.jpg', file_name): # jpgファイルのみ処理
    img_raw = cv2.imread(directory_name + file_name)
    # 画像をHSV形式へ変換
    img_hsv = cv2.cvtColor(img_raw, cv2.COLOR_BGR2HSV)
    # 2値化画像へ変換
    mask = cv2.inRange(img_hsv, lower, upper)
    # 最も広い白領域を残す
    # 白領域（連結している非ゼロの画素）ごとにラベルをつける
    num_Label, img_label = cv2.connectedComponents(mask)
    img_label = np.uint8(img_label)
    # ラベル0（黒）削除
    img_label_nonzero = img_label[img_label != 0]

    # 最も面積が広い白領域のラベル
    if len(img_label_nonzero)!= 0:
        big = stats.mode(img_label_nonzero)[0] # ラベル最頻値の取得
    else:
        big = 0
    #最も広い白領域のみ残す (bigの値をもった要素のみが255になる，ほかは0)
    img_dice = np.uint8(img_label == big)*255
    # サイコロ領域を矩形でクロップした画像の作成
    img_crop_bw, img_crop_rgb = cropimage(img_dice, img_raw)


    # 2値化（白黒）サイコロ画像の保存
    cv2.imwrite(directory_save_bw + 'bw-' + file_name, img_crop_bw)
    cv2.imwrite(directory_save_rgb + 'rgb-' + file_name, img_crop_rgb)
    # データ拡張用に画像を配列へ保管
    list_img_crop_bw[j] = img_crop_bw
    list_img_crop_rgb[j] = img_crop_rgb
    list_img_filename[j] = file_name
    j += 1

    # 読み込み枚数カウント
    dice_num = file_name[5] # サイコロ番号
    if dice_num == '1':
      dice1_count += 1
    elif dice_num == '2':
      dice2_count += 1
    elif dice_num == '3':
      dice3_count += 1
    elif dice_num == '4':
      dice4_count += 1
    elif dice_num == '5':
      dice5_count += 1
    elif dice_num == '6':
      dice6_count += 1

total_img_num = j # 読み込み枚数合計
print('Loaded images')
print('total: {}'.format(total_img_num))
print('dice1: {}'.format(dice1_count))
print('dice2: {}'.format(dice2_count))
print('dice3: {}'.format(dice3_count))
print('dice4: {}'.format(dice4_count))
print('dice5: {}'.format(dice5_count))
print('dice6: {}'.format(dice6_count))


['dice-1-20250514-110746.jpg', 'dice-1-20250514-110811.jpg', 'dice-1-20250514-110832.jpg', 'dice-1-20250514-110852.jpg', 'dice-1-20250514-110905.jpg', 'dice-2-20250514-110945.jpg', 'dice-2-20250514-111008.jpg', 'dice-2-20250514-111032.jpg', 'dice-2-20250514-111041.jpg', 'dice-2-20250514-111056.jpg', 'dice-3-20250514-111128.jpg', 'dice-3-20250514-111143.jpg', 'dice-3-20250514-111211.jpg', 'dice-3-20250514-111224.jpg', 'dice-3-20250514-111241.jpg', 'dice-4-20250514-111258.jpg', 'dice-4-20250514-111309.jpg', 'dice-4-20250514-111328.jpg', 'dice-4-20250514-111337.jpg', 'dice-4-20250514-111347.jpg', 'dice-5-20250514-111359.jpg', 'dice-5-20250514-111413.jpg', 'dice-5-20250514-111425.jpg', 'dice-5-20250514-111434.jpg', 'dice-5-20250514-111443.jpg', 'dice-6-20250514-111456.jpg', 'dice-6-20250514-111506.jpg', 'dice-6-20250514-111524.jpg', 'dice-6-20250514-111537.jpg', 'dice-6-20250514-111546.jpg', 'test']
File number: 31
Loaded images
total: 30
dice1: 5
dice2: 5
dice3: 5
dice4: 5
dice5: 5
dice6:

データ拡張を行い，学習用データを増やすプログラム

OpenCVを使います

In [ ]:
# データ拡張を行い，学習用画像を増やす

#回転角度の指定
angle = [0, 15, 30, 45, 60, 75, 90]

# 画像の回転処理
def rotate_image(img, angle):
  width, height = img.shape[:2] # 画像サイズ
  center = width/2, height/2
  Rotmat = cv2.getRotationMatrix2D(center=center, angle=angle, scale=1) # 回転行列
  img_rot = cv2.warpAffine(img, Rotmat, dsize=(width, height))
  return img_rot



# 保存先フォルダが存在しなかったら新規作成
# 白黒
directory_save_bw = '/content/drive/MyDrive/Colab Notebooks/AI-Kouza/dataset/img_dice_bw_rot/'
if not os.path.exists(directory_save_bw):
  os.mkdir(directory_save_bw)
# カラー
directory_save_rgb = '/content/drive/MyDrive/Colab Notebooks/AI-Kouza/dataset/img_dice_rgb_rot/'
if not os.path.exists(directory_save_rgb):
  os.mkdir(directory_save_rgb)

k = 0
for i in range(total_img_num):
  for j, deg in enumerate(angle):
    img_rot_bw = rotate_image(list_img_crop_bw[i], deg)
    img_rot_rgb = rotate_image(list_img_crop_rgb[i], deg)
    # 画像の保存
    cv2.imwrite(directory_save_bw + 'bw-' + str(list_img_filename[i])[2:-6] + '-' + str(deg) + '.jpg', img_rot_bw)
    cv2.imwrite(directory_save_rgb + 'rgb-' + str(list_img_filename[i])[2:-6] + '-' + str(deg) + '.jpg', img_rot_rgb)
    k += 1

print('Saved (BW): {}'.format(k))
print('Saved (COLOR): {}'.format(k))


Saved (BW): 210
Saved (COLOR): 210
